In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import platform
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go
from matplotlib import rcParams
import csv
import matplotlib.ticker as ticker
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import io
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지


total22_df = pd.read_csv("전연령22_utf8.csv", low_memory=False)
senior22_df = pd.read_csv("노인승하차22_utf8.csv", low_memory=False)
total24_df = pd.read_csv("전연령24_utf8.csv", low_memory=False)
senior24_df = pd.read_csv("노인승하차24_utf8.csv", low_memory=False)
total23_df = pd.read_csv("전연령23_utf8.csv", low_memory=False)
senior23_df = pd.read_csv("노인승하차23_utf8.csv", low_memory=False)
#senior_care_df = pd.read_csv("senior_care_utf8.csv", low_memory=False) # 파일이 사라짐
sme_df = pd.read_csv("sme.csv", low_memory=False)
park_df = pd.read_csv("TB_PTP_PRK_M.csv", low_memory=False)
safety_df = pd.read_csv("안전사고_utf.csv", low_memory=False)
elderly_2020_2022 = pd.read_csv(
    "elderly2020_2022.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'  # 문제 있는 라인은 건너뜀
)
elderly_2023_2025 = pd.read_csv(
    "elderly2023_2025.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'
)
weather_raw = pd.read_csv("weath20_25_utf8.csv", encoding='utf-8')
rain_raw = pd.read_csv("rain20_25_utf8.csv", encoding='utf-8') # 강수량 파일명 확인 필요
facilities_df = pd.read_csv("역별_편의시설.csv", low_memory=False)


In [16]:


# 1. 파일 불러오기 수정: header=1 추가 (첫 줄인 '승강기 관리현황'을 건너뛰고 두 번째 줄을 컬럼명으로 인식)
facilities_df = pd.read_csv("역별_편의시설.csv", low_memory=False, header=1)

# 2. '총 합계' 행 제거 및 NaN 제거
facilities_df = facilities_df[facilities_df['역명'].notna()]
facilities_df = facilities_df[facilities_df['역명'] != '총 합계']

# 3. 숫자 컬럼 전처리 (쉼표 제거 및 숫자 변환)
cols_to_sum = ['E/L', 'E/S', 'M/W', 'W/L', '수직형리프트']

for col in cols_to_sum:
    # 해당 컬럼이 데이터프레임에 있는지 확인 후 처리
    if col in facilities_df.columns:
        if facilities_df[col].dtype == 'object':
            facilities_df[col] = facilities_df[col].str.replace(',', '').astype(float).fillna(0)
        else:
            facilities_df[col] = facilities_df[col].fillna(0)

# 4. 역명 정제: 괄호와 숫자 제거 (예: '종로3가(1)' -> '종로3가')
facilities_df['역명_통합'] = facilities_df['역명'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()

# 5. 역명 기준으로 그룹화하여 합계 계산
final_facilities_df = facilities_df.groupby('역명_통합')[cols_to_sum].sum().reset_index()

# 결과 확인
print(final_facilities_df.head())

     역명_통합  E/L   E/S  M/W  W/L  수직형리프트
0     가락시장    9  34.0    0    0       0
1  가산디지털단지    4  16.0    0    3       0
2       강남    4   0.0    0    0       0
3     강남구청    2   7.0    0    0       0
4       강동    3  10.0    0    0       0


In [17]:


# 1. 분석하신 'Top 5 역' 리스트 (아까 궁금해하신 종로3가, 종로5가 포함 예시)
# 실제 분석 결과인 상위 5개 역 이름으로 이 부분을 꼭 수정하세요!
top5_list = ['종로3가', '청량리', '제기동', '연신내', '종로5가'] 

# 2. 전체 데이터에서 Top 5 역만 필터링 및 순서 정렬
df_plot = final_facilities_df[final_facilities_df['역명_통합'].isin(top5_list)].copy()
df_plot = df_plot.set_index('역명_통합').reindex(top5_list).reset_index()

# 3. Subplots 생성 (여기서는 하나의 큰 그래프에 묶어서 비교하므로 1행 1열)
fig = make_subplots(rows=1, cols=1, subplot_titles=("상위 5개 역 편의시설 현황"))

# 4. 각 시설물 별로 막대(Bar) 추가
facilities = ['E/L', 'E/S', 'W/L', '수직형리프트']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA'] # Plotly 기본 색상 팔레트 활용

for i, facility in enumerate(facilities):
    fig.add_trace(
        go.Bar(
            x=df_plot['역명_통합'],
            y=df_plot[facility],
            name=facility, # 범례 이름
            text=df_plot[facility], # 막대 위에 숫자 표시
            textposition='auto', # 숫자 위치 자동 조절
            marker_color=colors[i]
        ),
        row=1, col=1
    )

# 5. 레이아웃 설정 (막대를 그룹으로 묶어서 보여주기)
fig.update_layout(
    title_text="<b>수요 상위 역 편의시설 보유 현황</b>", # 전체 제목
    barmode='group', # 'group'으로 해야 막대가 옆으로 나란히 섭니다 (중요)
    xaxis_title="역명",
    yaxis_title="시설물 개수(대)",
    legend_title="시설 구분",
    template='plotly_white', # 깔끔한 흰색 배경
    height=600 # 그래프 높이
)

# 6. 그래프 출력
fig.show()